In [1]:
import pandas as pd
import numpy as np
import shapely
import matplotlib.pylab as plt
import geopandas as gpd
import fiona
import rtree

from geopandas import GeoDataFrame
from fiona.crs import from_epsg
from shapely.geometry import Point

%matplotlib inline

In [2]:
path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/Cleaned_DataSets/"
q_score = pd.read_csv(path + "streetscore_newyorkcity.csv")
q_score.head()

,latitude,longitude,q-score
0,40.700909,-74.013504,11.062166
1,40.701000,-74.013878,10.818611
2,40.701080,-74.012878,12.677955
3,40.701187,-74.013268,11.417325
4,40.701244,-74.012115,25.199091


In [3]:
crs = None
geometry = [Point(xy) for xy in zip(q_score.longitude, q_score.latitude)]
qscore_geo = GeoDataFrame(q_score, crs=crs, geometry=geometry)
qscore_geo.crs = fiona.crs.from_epsg(4326)

In [4]:
qscore_geo.head()

,latitude,longitude,q-score,geometry
0,40.700909,-74.013504,11.062166,POINT (-74.013504 40.700909)
1,40.701000,-74.013878,10.818611,POINT (-74.01387800000001 40.701)
2,40.701080,-74.012878,12.677955,POINT (-74.012878 40.70108)
3,40.701187,-74.013268,11.417325,POINT (-74.01326800000001 40.701187)
4,40.701244,-74.012115,25.199091,POINT (-74.01211500000001 40.701244)


In [5]:
shape_path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/si_mappluto_16v2/SIMapPLUTO.shp"
ct_shape = GeoDataFrame.from_file(shape_path)
ct_shape.to_crs(epsg=4326, inplace=True)

In [6]:
ct_shape.head()

,APPBBL,APPDate,Address,AreaSource,AssessLand,AssessTot,BBL,BldgArea,BldgClass,BldgDepth,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.000000e+00,None,534 CRAIG AVENUE,2,9978.0,25214.0,5.080500e+09,1472,A1,39.0,...,0,1915,None,10307,R1-2,None,None,None,35a,POLYGON ((-74.25036408443215 40.50913601637622...
1,0.000000e+00,None,HYLAN BOULEVARD,4,2004.0,2004.0,5.079390e+09,0,V0,0.0,...,0,0,None,10307,R3X,None,None,None,35a,POLYGON ((-74.25004640465613 40.50328123150459...
2,5.079230e+09,10/23/1992,258 CARTERET STREET,2,18485.0,50523.0,5.079230e+09,3475,B2,40.0,...,0,1992,None,10307,R3X,None,None,None,35a,POLYGON ((-74.24651575541495 40.50190473678119...
3,0.000000e+00,None,HYLAN BOULEVARD,4,4012.0,4012.0,5.079230e+09,0,V0,0.0,...,0,0,None,10307,R3X,None,None,None,35a,POLYGON ((-74.24738496730963 40.50324460474184...
4,0.000000e+00,None,59 SATTERLEE STREET,2,11224.0,32678.0,5.079450e+09,1852,B2,46.0,...,0,1975,None,10307,R3X,None,None,None,35a,POLYGON ((-74.25275072116283 40.50589254047246...


In [7]:
ct_shape.columns

Index([    u'APPBBL',    u'APPDate',    u'Address', u'AreaSource',
       u'AssessLand',  u'AssessTot',        u'BBL',   u'BldgArea',
        u'BldgClass',  u'BldgDepth',  u'BldgFront',      u'Block',
         u'BoroCode',    u'Borough',   u'BsmtCode',   u'BuiltFAR',
           u'CB2010',         u'CD',     u'CT2010',    u'ComArea',
          u'CommFAR',    u'CondoNo',    u'Council',  u'EDesigNum',
        u'Easements', u'ExemptLand',  u'ExemptTot',        u'Ext',
         u'FacilFAR', u'FactryArea',   u'FireComp', u'GarageArea',
       u'HealthArea',   u'HistDist', u'IrrLotCode',    u'LandUse',
         u'Landmark',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType',  u'LtdHeight', u'MAPPLUTO_F',
         u'NumBldgs',  u'NumFloors', u'OfficeArea',  u'OtherArea',
         u'Overlay1',   u'Overlay2',  u'OwnerName',  u'OwnerType',
       u'PLUTOMapID', u'PolicePrct',   u'ProxCode',    u'ResArea',
         u'ResidFAR', u'RetailArea', u'SHAPE_Area', u'SHAPE_Le

In [8]:
SI_Shape = ct_shape.drop([u'Borough', u'CD', u'CB2010', u'Council', u'SanitBoro', u'SanitDistr', u'SanitSub', 
                          u'Overlay1',u'Overlay2', u'SPDist1', u'SPDist2', u'SPDist3', u'LtdHeight', u'SplitZone', 
                          u'Easements', u'OwnerName', u'AreaSource', u'IrrLotCode', u'BsmtCode', u'AssessLand', 
                          u'AssessTot', u'ExemptLand', u'ExemptTot', u'YearAlter1', u'YearAlter2',u'BuiltFAR', 
                          u'ResidFAR', u'CommFAR', u'FacilFAR', u'BoroCode', u'BBL', u'CondoNo', u'Tract2010', 
                          u'ZoneMap', u'ZMCode', u'Sanborn', u'TaxMap', u'EDesigNum', u'APPBBL', u'APPDate', 
                          u'PLUTOMapID', u'Version', u'Ext', u'HistDist', u'Landmark', u'ZoneDist2', u'ZoneDist3', 
                          u'ZoneDist4'], axis = 1)

In [9]:
SI_Shape.columns

Index([   u'Address',   u'BldgArea',  u'BldgClass',  u'BldgDepth',
        u'BldgFront',      u'Block',     u'CT2010',    u'ComArea',
       u'FactryArea',   u'FireComp', u'GarageArea', u'HealthArea',
          u'LandUse',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'OfficeArea',  u'OtherArea',  u'OwnerType',
       u'PolicePrct',   u'ProxCode',    u'ResArea', u'RetailArea',
       u'SHAPE_Area', u'SHAPE_Leng', u'SchoolDist',  u'StrgeArea',
         u'UnitsRes', u'UnitsTotal',     u'XCoord',     u'YCoord',
        u'YearBuilt',    u'ZipCode',  u'ZoneDist1',   u'geometry'],
      dtype='object')

In [10]:
SI_Shape_2 = SI_Shape.drop([u'Address', u'BldgClass', u'BldgDepth', u'ComArea', u'FactryArea',
              u'FireComp', u'GarageArea', u'HealthArea', u'Lot',u'LotDepth',u'LotFront',
              u'LotType',u'OfficeArea',  u'OtherArea',  u'OwnerType',u'PolicePrct',   
              u'ProxCode',    u'ResArea', u'RetailArea',u'SHAPE_Leng', u'SchoolDist',  
              u'StrgeArea', u'UnitsRes',u'ZoneDist1'],axis = 1)
SI_Shape_2.columns

Index([  u'BldgArea',  u'BldgFront',      u'Block',     u'CT2010',
          u'LandUse',    u'LotArea', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'SHAPE_Area', u'UnitsTotal',     u'XCoord',
           u'YCoord',  u'YearBuilt',    u'ZipCode',   u'geometry'],
      dtype='object')

In [11]:
SI_Shape_2.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry
0,1472,16.0,8050,248,01,3828,0,2,2.5,3533.913589,1,914595,124882,1915,10307,POLYGON ((-74.25036408443215 40.50913601637622...
1,0,0.0,7939,244.01,11,1000,0,0,0.0,943.619886,0,0,0,0,10307,POLYGON ((-74.25004640465613 40.50328123150459...
2,3475,53.0,7923,244.02,01,7215,0,1,2.0,7246.669304,2,915635,122253,1992,10307,POLYGON ((-74.24651575541495 40.50190473678119...
3,0,0.0,7923,244.02,11,2000,0,0,0.0,1941.861988,0,0,0,0,10307,POLYGON ((-74.24738496730963 40.50324460474184...
4,1852,27.0,7945,244.01,01,3800,0,1,2.0,3816.074809,2,913913,123674,1975,10307,POLYGON ((-74.25275072116283 40.50589254047246...


In [12]:
SI_Shape_joined = gpd.sjoin(SI_Shape_2, qscore_geo, how="inner", op='intersects')
SI_Shape_joined.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry,index_right,latitude,longitude,q-score
7,622,20.0,7929,244.01,01,7400,0,1,1.5,7356.345387,1,915080,124082,1925,10307,POLYGON ((-74.24855382585523 40.50697245751223...,139768,40.506947,-74.248550,21.176020
12,1949,48.0,7926,244.01,01,6751,0,1,1.0,6607.890594,1,915213,123500,1983,10307,POLYGON ((-74.24807045752007 40.50537545624927...,139585,40.505322,-74.248398,29.015736
21,3080,36.0,7924,244.02,01,4875,0,1,2.0,4930.127539,2,915296,122541,1993,10307,POLYGON ((-74.24775252347756 40.50272195363797...,139357,40.502731,-74.248016,24.293695
22,3500,50.0,7927,244.01,01,6305,0,1,2.0,6417.433811,2,914954,123471,1988,10307,POLYGON ((-74.24902260147118 40.50536958866473...,139583,40.505318,-74.249222,23.498457
24,3020,46.0,7925,244.02,01,5850,0,1,2.0,5963.721467,2,915089,121938,1994,10307,POLYGON ((-74.24849218034011 40.50107233599633...,139236,40.501114,-74.248535,24.447577


In [13]:
SI_Shape_joined_2 = gpd.sjoin(qscore_geo, SI_Shape_2, how="inner", op='intersects')
SI_Shape_joined_2.head()

,latitude,longitude,q-score,geometry,index_right,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode
139189,40.499561,-74.244972,25.401331,POINT (-74.244972 40.499561),991,1875,40.0,7857,244.02,09,4696428,0,2,1.0,4.141297e+06,1,916032,120308,1940,10307
139197,40.500305,-74.245018,29.385057,POINT (-74.245018 40.500305),991,1875,40.0,7857,244.02,09,4696428,0,2,1.0,4.141297e+06,1,916032,120308,1940,10307
139198,40.500317,-74.245384,26.838640,POINT (-74.245384 40.500317),991,1875,40.0,7857,244.02,09,4696428,0,2,1.0,4.141297e+06,1,916032,120308,1940,10307
139202,40.500423,-74.248489,28.845238,POINT (-74.24848900000001 40.500423),991,1875,40.0,7857,244.02,09,4696428,0,2,1.0,4.141297e+06,1,916032,120308,1940,10307
139203,40.500435,-74.248878,29.749092,POINT (-74.248878 40.500435),991,1875,40.0,7857,244.02,09,4696428,0,2,1.0,4.141297e+06,1,916032,120308,1940,10307


In [14]:
SI_Shape_joined.to_csv('SI_Geometry_Joined.csv')

In [15]:
SI_Shape_joined_2.to_csv('SI_Point_Joined.csv')